<a href="https://colab.research.google.com/github/laddhashreya2000/AI-Email-Classifier/blob/main/code/Model_building_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting up the Root Dir.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# change this path to the cloned git repository
%cd /content/drive/My Drive/Project_Softie


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1aiL_ODB7FUeFLuLx8a2ZnWgmpA43A_uw/Project_Softie


In [ ]:
!pip install scikit-learn --upgrade

     |████████████████████████████████| 6.8MB 6.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


#Importing libraries

In [ ]:
# First we import the required libraries

import tensorflow as tf
import random
import numpy as np
import pandas as pd
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import io
import sklearn

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K 
K.clear_session()

# Check tensorflow version
if float(tf.__version__[0]) < 2.0:
  print('Updating tensorflow')
  !pip install tensorflow==2.0
else:
  print('Correct version of Tensorflow installed.')

Correct version of Tensorflow installed.


#Reading the cleaned/preprocessed data

In [ ]:
data = pd.read_csv('./files/dataset_processed.csv')
data.fillna('')
data.count()
data.head()

,Unnamed: 0,Unnamed: 0.1,label,text,t1
0,0,0,retirements,request information monday november pm pension...,request information monday november pm pension...
1,1,1,mdu,pension scheme friday november pm rbc pension ...,pension scheme friday november pm rbc pension ...
2,2,2,mdu,change address scheme november change address ...,change address scheme november change address ...
3,3,3,transfers,pension fund ref friday november pm pension fu...,pension fund ref friday november pm pension fu...
4,4,4,transfers,zzz friday november pm pension fund zzz dear s...,zzz friday november pm pension fund zzz dear s...


In [ ]:
# encoding the labels
possible_labels = data.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'mdu': 1, 'retirements': 0, 'transfers': 2}

In [ ]:
data['label'] = data.label.replace(label_dict)

#Splitting the Dataset into Training and Testing Dataset

In [ ]:
# split = 0.15
# split_n = int(round(len(data['t1'])*(1-split),0))
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data['t1'], data['label'], test_size=0.15, random_state=42, stratify=data['label'])
#train_data = padded[:split_n]
#train_labels = data['label'].values[:split_n]
#test_data = padded[split_n:]
#test_labels = data['label'].values[split_n:]

#Naive Bayes using Pre-Defined Libraries

In [ ]:
#Naive-Bayes classification
from sklearn.feature_extraction.text import CountVectorizer
pipe3 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('model', MultinomialNB(alpha = 100))])

model_nb = pipe3.fit(train_data, train_labels)
nb_pred = model_nb.predict(test_data)

print("Accuracy of Naive Bayes Classifier: {}%".format(round(accuracy_score(test_labels, nb_pred)*100,2)))
print("\nConfusion Matrix of Naive Bayes Classifier:\n")
print(confusion_matrix(test_labels, nb_pred))
print("\nClassification Report of Naive Bayes Classifier:\n")
print(classification_report(test_labels, nb_pred))


Accuracy of Naive Bayes Classifier: 84.62%

Confusion Matrix of Naive Bayes Classifier:

[[32  1  1]
 [ 4 31  0]
 [10  0 25]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

           0       0.70      0.94      0.80        34
           1       0.97      0.89      0.93        35
           2       0.96      0.71      0.82        35

    accuracy                           0.85       104
   macro avg       0.88      0.85      0.85       104
weighted avg       0.88      0.85      0.85       104



In [ ]:
# export the model which has the best accuracy
import pickle
pkl_filename = "./models/pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model_nb, file)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'model__alpha' : [1, .1, .01, .001, .0001, 10, 100, 1000]}

grid1 = GridSearchCV(pipe3,parameters,refit = True, verbose=0)

grid1.fit(train_data,train_labels)



predic = grid1.predict(test_data)
print(classification_report(test_labels,predic))
print(confusion_matrix(test_labels, predic))

              precision    recall  f1-score   support

           0       0.92      0.65      0.76        34
           1       0.85      1.00      0.92        35
           2       0.82      0.91      0.86        35

    accuracy                           0.86       104
   macro avg       0.86      0.85      0.85       104
weighted avg       0.86      0.86      0.85       104

[[22  5  7]
 [ 0 35  0]
 [ 2  1 32]]


In [ ]:
print(grid1.best_params_)

{'model__alpha': 1}


# Logistic Regression with Hashing Vectorizer

In [ ]:

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LogisticRegression

pipe4 = Pipeline([('vect', HashingVectorizer()), ('tfidf', TfidfTransformer()), ('model', LogisticRegression(random_state=0))])

model_lr = pipe4.fit(train_data, train_labels)
lr_pred = model_lr.predict(test_data)

print("Accuracy of Logistic Regression: {}%".format(round(accuracy_score(test_labels, lr_pred)*100,2)))
print("\nConfusion Matrix of Logistic Regression:\n")
print(confusion_matrix(test_labels, lr_pred))
print("\nClassification Report of Logistic Regression:\n")
print(classification_report(test_labels, lr_pred))


Accuracy of Naive Bayes Classifier: 91.35%

Confusion Matrix of Naive Bayes Classifier:

[[27  3  4]
 [ 0 35  0]
 [ 2  0 33]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

           0       0.93      0.79      0.86        34
           1       0.92      1.00      0.96        35
           2       0.89      0.94      0.92        35

    accuracy                           0.91       104
   macro avg       0.91      0.91      0.91       104
weighted avg       0.91      0.91      0.91       104



# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipe5 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('model', RandomForestClassifier(max_depth =9, random_state = 42))])

model_rf = pipe5.fit(train_data, train_labels)
nb_pred = model_rf.predict(test_data)

print("Accuracy of Random Forest Classifier: {}%".format(round(accuracy_score(test_labels, nb_pred)*100,2)))
print("\nConfusion Matrix of Random Forest Classifier:\n")
print(confusion_matrix(test_labels, nb_pred))
print("\nClassification Report of Random Forest Classifier:\n")
print(classification_report(test_labels, nb_pred))

Accuracy of Naive Bayes Classifier: 87.5%

Confusion Matrix of Naive Bayes Classifier:

[[22  6  6]
 [ 0 35  0]
 [ 1  0 34]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

           0       0.96      0.65      0.77        34
           1       0.85      1.00      0.92        35
           2       0.85      0.97      0.91        35

    accuracy                           0.88       104
   macro avg       0.89      0.87      0.87       104
weighted avg       0.89      0.88      0.87       104



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__bootstrap': [True],
    'model__max_depth': [3],
    #'model__max_features': [100],
    #'model__min_samples_leaf': [3, 4, 5],
    # 'model__min_samples_split': [8, 10, 12],
    'model__n_estimators': [100]
}

grid = GridSearchCV(pipe5,param_grid,refit = True)

grid.fit(train_data,train_labels)



predic = grid.predict(test_data)
print(classification_report(test_labels,predic))
print(confusion_matrix(test_labels, predic))

              precision    recall  f1-score   support

           0       0.96      0.71      0.81        34
           1       0.89      0.94      0.92        35
           2       0.83      1.00      0.91        35

    accuracy                           0.88       104
   macro avg       0.90      0.88      0.88       104
weighted avg       0.89      0.88      0.88       104

[[24  4  6]
 [ 1 33  1]
 [ 0  0 35]]


#K-Fold Cross Validation on Naive Bayes

In [ ]:
from sklearn.model_selection import KFold


X=data['text'].values
y=data['label'].values

acc = []

kf= KFold(n_splits=5)
for train_idx, test_idx in kf.split(X,y):
    X_train= X[train_idx]
    y_train= y[train_idx]
    X_valid= X[test_idx]
    y_valid= y[test_idx]
    model_nb = pipe3.fit(X_train, y_train)
    nb_pred = model_nb.predict(X_valid)
    acc.append(accuracy_score(y_valid, nb_pred))


print(np.mean(acc))

0.46796610169491526


In [ ]:
#Plot for Naive Bayes(using pre-defined libraries)
#  title = "Learning Curves (Naive Bayes)"
 # Cross validation with 5 splits to get smoother mean test and train
 # score curves, each time with 35% data randomly selected as a validation set.
#  cv = ShuffleSplit(n_splits=5, test_size=0.35, random_state=0)

#  estimator = pipe3
# plot_learning_curve(estimator, title, data['text'].values, data['label'].values, ylim=(0.7, 1.01), cv=cv, n_jobs=4)

# plt.show()

#SVM Implementation using Pre-Defined Libraries

In [ ]:
#Support Vector classification


pipe2 = Pipeline([('vect', HashingVectorizer()), ('tfidf', TfidfTransformer()), ('model', LinearSVC(C=.1))])

model_svc = pipe2.fit(train_data, train_labels)
svc_pred = model_svc.predict(test_data)

print("Accuracy of SVM Classifier: {}%".format(round(accuracy_score(test_labels, svc_pred)*100,2)))
print("\nConfusion Matrix of SVM Classifier:\n")
print(confusion_matrix(test_labels, svc_pred))
print("\nClassification Report of SVM Classifier:\n")
print(classification_report(test_labels, svc_pred))

Accuracy of SVM Classifier: 95.56%

Confusion Matrix of SVM Classifier:

[[13  0  2]
 [ 0 15  0]
 [ 0  0 15]]

Classification Report of SVM Classifier:

              precision    recall  f1-score   support

           0       1.00      0.87      0.93        15
           1       1.00      1.00      1.00        15
           2       0.88      1.00      0.94        15

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



In [ ]:
sklearn_version = sklearn.__version__

print(sklearn_version)

0.23.2


In [ ]:
from sklearn.svm import SVC


pipe2 = Pipeline([('vect', HashingVectorizer()), ('tfidf', TfidfTransformer()), ('model', SVC(kernel='linear', gamma='auto', C=1))])




model_svc = pipe2.fit(train_data, train_labels)
svc_pred = model_svc.predict(test_data)

print("Accuracy of SVM Classifier: {}%".format(round(accuracy_score(test_labels, svc_pred)*100,2)))
print("\nConfusion Matrix of SVM Classifier:\n")
print(confusion_matrix(test_labels, svc_pred))
print("\nClassification Report of SVM Classifier:\n")
print(classification_report(test_labels, svc_pred))

Accuracy of SVM Classifier: 91.11%

Confusion Matrix of SVM Classifier:

[[12  1  2]
 [ 1 14  0]
 [ 0  0 15]]

Classification Report of SVM Classifier:

              precision    recall  f1-score   support

           0       0.92      0.80      0.86        15
           1       0.93      0.93      0.93        15
           2       0.88      1.00      0.94        15

    accuracy                           0.91        45
   macro avg       0.91      0.91      0.91        45
weighted avg       0.91      0.91      0.91        45



#Hyper Parameters Determination using Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'model__C': [0.1, 1, 10, 100, 1000],  
              'model__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'model__kernel': ['rbf', 'linear']}  

grid = GridSearchCV(pipe2,param_grid,refit = True, verbose=0)

grid.fit(train_data,train_labels)



predic = grid.predict(test_data)
print(classification_report(test_labels,predic))
print(confusion_matrix(test_labels, predic))


              precision    recall  f1-score   support

           0       0.92      0.80      0.86        15
           1       0.93      0.93      0.93        15
           2       0.88      1.00      0.94        15

    accuracy                           0.91        45
   macro avg       0.91      0.91      0.91        45
weighted avg       0.91      0.91      0.91        45

[[12  1  2]
 [ 1 14  0]
 [ 0  0 15]]


In [ ]:
print(grid.best_params_)

{'model__C': 1, 'model__gamma': 1, 'model__kernel': 'linear'}


In [ ]:
import pickle
pkl_filename = "./models/pickle_model_svc.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model_svc, file)